In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import requests

In [2]:
class Station:
    def __init__(self, nom:str, capteur:int, apiLabel:str) -> None:
        self.nom:str = nom
        self.capteur:int = capteur
        self.apiLabel:str = apiLabel


class Record:
    def __init__(self, temperature:float, humidite:int, pression:int, heure:datetime, station:Station) -> None:
        self.temperature:float = temperature
        self.humidite:int = humidite
        self.pression:int = pression
        self.heure:datetime = heure
        self.station:Station = station


class IDataExtractor:
    @staticmethod
    def extract() -> dict:
        raise NotImplementedError("This function must be implemented.")
        
        
class CSVExtractor(IDataExtractor):
    @staticmethod
    def extract(filename: str):
        return pd.read_csv(filename, sep=";")


class IAPIDataExtractor(IDataExtractor):
    @staticmethod
    def extract(route: str):
        raise NotImplementedError("This function must be implemented.")


class APIStations(IAPIDataExtractor):
    @staticmethod
    def extract():
        url = "https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/stations-meteo-en-place/records?select=id_nom%2C%20id_numero%2C%20ville&order_by=id_numero&limit=80"
        response = requests.get(url)
        stations = response.json()
        return pd.DataFrame(stations['results'])


class IPrinting:
    @staticmethod
    def print() -> None:
        raise NotImplementedError("This function must be implemented")
    

class RecordPrinting(IPrinting):
    @staticmethod
    def print(record: Record):
        print(f"{record.heure}\nTempérature : {record.temperature} ; Humidité : {record.humidite} ; Pression : {record.pression}")


class StationPrinting(IPrinting):
    @staticmethod
    def print(station: Station):
        url = f"https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/{station.apiLabel}/records?select=data%2C%20id%2C%20humidite%2C%20pression%2C%20temperature_en_degre_c%2C%20heure_de_paris&order_by=-heure_utc&limit=1"
        response = requests.get(url)
        data = response.json()
        if not data.get('results'):
            print("Aucun résultat trouvé pour cette station.")
            return
        data = pd.DataFrame(data['results']).iloc[0] # première ligne
        record = Record(data["temperature_en_degre_c"], data["humidite"], data["pression"], data["heure_de_paris"], station)
        RecordPrinting.print(record)


class BatchStationsPrinting(IPrinting):
    @staticmethod
    def print(stations: list[Station]):
        for i in range(stations.shape[0]):
            station = stations.iloc[i]
            print(station['id_nom'][26:].center(60, "-"))
            stat = Station(station['id_nom'][11:], station['id_numero'], station['id_nom'])
            StationPrinting.print(stat)


class AllStationsPrinting(BatchStationsPrinting, APIStations):
    def __init__(self) -> None:
        self.print(self.extract())


class RecordExtractor:
    @staticmethod
    def extract(data) -> Record:
        temperature = data.get("temperature")
        humidite = data.get("humidite")
        pression = data.get("pression")
        heure = data.get("heure_de_paris")
        station = data.get("id")
        return Record(temperature, humidite, pression, heure, station)


class Test:
    @staticmethod
    def test() -> None:
        raise NotImplementedError("This function must be implemented.")


class TestMissing(Test):
    @staticmethod
    def test(line) -> None:
        record = RecordExtractor.extract(line)
        fields = ["temperature", "humidite", "pression", "heure", "station"]
        #passed = True
        for field in fields:
            valeur = getattr(record, field)
            #assert pd.notnull(valeur), f"Le champ {field} est nul dans la ligne extraite!"
            if (not pd.notnull(valeur)):
                #print(f"Le champ {field} est nul dans la ligne!")
                #passed = False
                return False
        #if not passed:
        #    print(f"Test échoué : ligne {i+2}.")
        return True


class TestBatch(Test):
    @staticmethod
    def test(data):
        for i in range(data.shape[0]):
            if not TestMissing.test(data.iloc[i]):
                #print(f"Erreur ligne {i}")
                return False
            else:
                return True


class Clean:
    @staticmethod
    def getCleanOnly(data: pd.DataFrame):
        """
        Permet de récupérer les lignes ne contenant pas de valeur manquante et les colonnes d'enregistrement
        en utilisant TestBatch pour tester chaque ligne.
        """
        # Construit un masque pour les lignes qui passent le test de TestBatch
        mask = [TestMissing.test(data.iloc[i]) for i in range(data.shape[0])]
        return data[mask]


class Configuration:
    def __init__(self) -> None:
        pass


class StationSelector(Configuration):
    def __init__(self) -> None:
        super().__init__()


class Ville:
    def __init__(self) -> None:
        pass


# Ajouter la visualisation, le nettoyage(?), le stockage, la transformation, l'extraction, les tests, la config initiale (inputs)
# StationSelector, DataVisualizer

In [3]:
AllStationsPrinting()

---------------------------valade---------------------------
2022-07-26T12:15:00+00:00
Température : 25.1 ; Humidité : 47 ; Pression : 99900
-------------------------meteopole--------------------------
12/9/2025 23:45
Température : -50.0 ; Humidité : 0 ; Pression : 90000
--------------------------marengo---------------------------
2025-11-17T23:00:00+00:00
Température : 6.6 ; Humidité : 77 ; Pression : 99800
---------------------------busca----------------------------
2025-08-25T07:45:00+00:00
Température : 21.7 ; Humidité : 64 ; Pression : 99700
------------------------ile-empalot-------------------------
2024-02-11T07:00:00+00:00
Température : 7.1 ; Humidité : 85 ; Pression : 98600
------------------------pont-Banleve------------------------
Aucun résultat trouvé pour cette station.
------------------------LIFE-hall-1-------------------------
Aucun résultat trouvé pour cette station.
--------------------------nakache---------------------------
2025-07-08T07:15:00+00:00
Température : 

In [ ]:
# liste chaînée pour afficher la météo station par station
# liste chaînée pour extraire les stations
# un dictionnaire en guise de fichier de configuration

In [4]:
TestBatch.test(data)

NameError: name 'data' is not defined

In [ ]:
import functools
import hashlib
import json
import time

class MemoryCache:
    """
    Système simple de cache en mémoire pour les résultats de fonction.
    Permet de contrôler l'expiration du cache (mise à jour périodique)
    """
    def __init__(self, ttl_seconds=600):
        """
        Paramètres :
            ttl_seconds : durée en secondes avant qu'une entrée du cache n'expire (time-to-live).
        """
        self.ttl = ttl_seconds
        self.cache = {}

    def get_cache_key(self, args, kwargs):
        try:
            # On génère une clé unique basée sur les arguments
            key = hashlib.md5(
                (
                    json.dumps(args, sort_keys=True, default=str) +
                    json.dumps(kwargs, sort_keys=True, default=str)
                ).encode()
            ).hexdigest()
        except Exception:
            key = str(args) + str(kwargs)  # fallback si erreur de sérialisation
        return key

    def cache_result(self, func):
        """
        Décorateur pour mettre en cache le résultat de la fonction décorée.
        On stocke le résultat ET l'heure de mise en cache.
        """
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = self.get_cache_key(args, kwargs)
            now = time.time()
            # Vérifier présence dans le cache et validité temporelle
            if key in self.cache:
                value, timestamp = self.cache[key]
                if (now - timestamp) < self.ttl:
                    # Données encore valides, utiliser le cache
                    return value
                # Sinon, supprimées et on refait la requête ci-dessous

            # Sinon ou périmé, effectuer la fonction (la requête par ex)
            result = func(*args, **kwargs)
            self.cache[key] = (result, now)
            return result

        return wrapper

    def clear(self):
        """Vide totalement le cache."""
        self.cache.clear()

# --- Commentaire expliquant le fonctionnement ---
"""
Fonctionnement :
- Lorsqu'une fonction (ex: get_url) est décorée par ce système de cache, il intercepte les appels.
- À chaque appel, il vérifie si une entrée correspondant aux mêmes arguments existe déjà dans le cache ET si elle n'a pas expiré (TTL).
- Si OUI (donnée présente et 'fraîche'), le résultat déjà en cache est retourné immédiatement, et la requête réelle n'est pas refaite.
- Si NON (pas encore en cache, arguments différents, ou cache périmé), la fonction est appelée normalement (ex: la requête HTTP est refaite) puis le résultat est stocké dans le cache associé à l'heure d'insertion.
- Pour forcer une mise à jour (par exemple : expiration du cache, ou après l'appel à `.clear()`), la fonction refait la récupération et réécrit la valeur en cache.

Gestion de la "mise à jour" :
 - Il suffit d'attendre que le TTL ("time to live") expire, ou d'appeler explicitement `clear()` sur l'objet de cache pour rafraîchir les données lors du prochain appel.

"""

# Exemple pratique d'utilisation sur une requête HTTP :
memory_cache = MemoryCache(ttl_seconds=600)  # 10 minutes

@memory_cache.cache_result
def get_url_cached(url):
    response = requests.get(url)
    return response.json()  # ou response.text selon le besoin

# Exemple d'utilisation :
# stations = pd.DataFrame(get_url_cached(url)['results'])

# Pour rafraîchir explicitement le cache :
# memory_cache.clear()




In [ ]:
# Système de cache simple pour les résultats des requêtes

# On utilise un simple dictionnaire pour mettre en cache les réponses
requete_cache = {}

def requete_avec_cache(url):
    if url in requete_cache:
        # Retourner le résultat du cache si déjà présent
        print("Résultat récupéré depuis le cache.")
        return requete_cache[url]
    # Sinon, effectuer la requête réelle
    response = requests.get(url)
    result = response.json()
    # Stocker le résultat dans le cache
    requete_cache[url] = result
    print("Résultat récupéré depuis l'API et mis en cache.")
    return result

# Exemple d'utilisation :
# data = requete_avec_cache("https://url_de_votre_api")

